# Citeseer Example

## Data

In [1]:
metadata ={
    "description": "CITESEER dataset.",
    "data": {
        "Node": {
            "NodeFeature": {
                "description": "Node features of Citeseer dataset.",
                "type": "float",
                "format": "SparseTensor",
                "file": "citeseer.npz",
                "key": "node_feats"
            },
            "NodeLabel": {
                "description": "Node labels of Citeseer dataset, int ranged from 1 to 6.",
                "type": "int",
                "format": "Tensor",
                "file": "citeseer.npz",
                "key": "node_class",
                "num_classes": 6
            }
        },
        "Edge": {
            "_Edge": {
                "file": "citeseer.npz",
                "key": "edge"
            }
        },
        "Graph": {
            "_NodeList": {
                "file": "citeseer.npz",
                "key": "node_list"
            },
            "_EdgeList": {
                "file": "citeseer.npz",
                "key": "edge_list"
            }
        }
    },
    "citation": "@inproceedings{yang2016revisiting,\ntitle={Revisiting semi-supervised learning with graph embeddings},\nauthor={Yang, Zhilin and Cohen, William and Salakhudinov, Ruslan},\nbooktitle={International conference on machine learning},\npages={40--48},\nyear={2016},\norganization={PMLR}\n}"
 }
 

In [2]:
import json
import numpy as np
import torch
import scipy.sparse as sparse
from dgl.data import CiteseerGraphDataset
dataset = CiteseerGraphDataset()
graph = dataset[0]

Using backend: pytorch


Extracting file to /home/jimmyzxj/.dgl/citeseer
Finished data loading and preprocessing.
  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


/home/jimmyzxj/miniconda3/envs/py39/lib/python3.9/site-packages/dgl/data/citation_graph.py:275: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


We can save the data file in only one file `citeseer.npz`.

In [3]:
# citeseer_feat
node_feats = sparse.csr_matrix(graph.ndata["feat"].numpy())
# citeseer_class
node_class = graph.ndata["label"].numpy()  # (3327,)
# citeseer_edge
edge = torch.stack(graph.edges()).numpy().T
# citeseer only has 1 single graph, there are some isolated nodes in the graph. 
# These isolated nodes are added as zero-vecs into the right position
node_list = np.ones((1, graph.num_nodes()))  # (1, 3327)
edge_list = np.ones((1, graph.num_edges()))  # (1, 9228)

data = {
    "node_feats": node_feats,
    "node_class": node_class,
    "edge": edge,
    "node_list": node_list,
    "edge_list": edge_list
}

In [4]:
np.savez_compressed("citeseer.npz", **data)

In [5]:
citeseer = np.load("citeseer.npz", allow_pickle=True)
citeseer.files

['node_feats', 'node_class', 'edge', 'node_list', 'edge_list']

In [6]:
!du citeseer.npz -h

208K	citeseer.npz


In [7]:
json.dump(metadata, open("metadata.json", "w"))

## Task

In [8]:
task = {
    "description": "Node classification on CITESEER dataset. Planetoid split.",
    "type": "NodeClassification",
    "feature": [{
        "object": "Node",
        "attribute": "NodeFeature"
    }],
    "target": {
        "object": "Node",
        "attribute": "NodeLabel",
        "num_classes": 6
    },
    "train_set": {
        "file": "citeseer_task.npz",
        "key": "train"
    },
    "val_set": {
        "file": "citeseer_task.npz",
        "key": "val"
    },
    "test_set": {
        "file": "citeseer_task.npz",
        "key": "test"
    }
}


In [9]:
train_set = graph.ndata["train_mask"].nonzero().squeeze().numpy()
val_set = graph.ndata["val_mask"].nonzero().squeeze().numpy()
test_set = graph.ndata["test_mask"].nonzero().squeeze().numpy()

In [10]:
task_data = {
    "train": train_set,
    "val": val_set,
    "test": test_set
}
np.savez_compressed("citeseer_task.npz", **task_data)


